In [ ]:
import os
import warnings
import logging
from time import sleep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pynq import PL, Overlay, allocate, DefaultIP

# Configure logging
log = logging.getLogger("CZT_Driver")
logging.basicConfig(level=logging.INFO)

# Command definitions
_command_valids = [0xE0, 0xE1, 0xE2, 0xE3, 0xE4, 0xE5, 0xE6, 0xE7, 0xE8, 0xE9, 0x9D, 0x9E, 0x86, 0xA3, 0x96, 0x9A, 0x02, 0x85, 0x05, 0x8C, 0x21, 0xA1, 0x1F, 0x9F, 0x20, 0xA0, 0x81, 0x01, 0x32, 0xB2, 0x34, 0xB4, 0x07, 0x87, 0x0B, 0x8B, 0x03, 0x83, 0x84, 0x04, 0xCB, 0x10, 0x90, 0x48, 0xC8, 0x4A, 0xCA]
_command_has_reply = [0xE0, 0x9D, 0x9E, 0x86, 0xA3, 0x96, 0x9A, 0xA1, 0x9F, 0xA0, 0xB2, 0xB4, 0x87, 0x8B, 0xC8, 0x83, 0x84, 0xCB]
_command_has_data = [0x21, 0x1F, 0x20, 0x32, 0x07, 0x0B, 0x48, 0x03, 0x04]
_command_map = {
    "READ_PART_BASE": 0xE0, "READ_SERIAL_LSB": 0x9D, "READ_SERIAL_MSB": 0x9E,
    "READ_FIRMWARE_VERSION": 0x86, "READ_MODULE_VERSION": 0xA3, "READ_STATUS": 0x96,
    "READ_TEMPERATURE": 0x9A, "BREAK": 0x02, "EVENT_ON": 0x85, "EVENT_OFF": 0x05,
    "FIFO_CLEAR": 0x8C, "SET_THRESHOLD": 0x21, "GET_THRESHOLD": 0xA1,
    "SET_GPIO": 0x1F, "GET_GPIO": 0x9F, "SET_CLOCK": 0x20, "GET_CLOCK": 0xA0,
    "RESTORE_SETUP": 0x81, "UPDATE_SETUP": 0x01, "SET_PEAKING_TIME": 0x32,
    "GET_PEAKING_TIME": 0xB2, "RUN_SELF_TEST": 0x34, "GET_SELF_TEST_RESULTS": 0xB4,
    "SET_CHANNEL": 0x07, "GET_CHANNEL": 0x87, "CHANNEL_CONTROL": 0x0B,
    "CHANNEL_STATUS": 0x8B, "SET_ADDR_POINTER": 0x03, "GET_ADDR_POINTER": 0x83,
    "READ_RAM": 0x84, "WRITE_RAM": 0x04, "EEPROM_CHECKSUM": 0xCB,
    "HOLD_ON": 0x10, "HOLD_OFF": 0x90, "SET_EMULATOR": 0x48, "GET_EMULATOR": 0xC8,
    "WRITE_PROTECT_OFF": 0x4A, "WRITE_PROTECT_ON": 0xCA
}

class CZTDriver(DefaultIP):
    """Class for interacting with the AXI CZT Controller version 1."""
    bindto = ['user.org:user:AXI_CZT_Controller:1.0']

    def __init__(self, description):
        super().__init__(description=description)
        self.is_channel_disabled = [0] * 256
        self.disabled_channel_list = []
        self.num_disabled = 0

    def command(self, com, inp=None, timeout=5000):
        self.write(0x0, 0x0000)  # Reset command valid

        if isinstance(com, str):
            com = _command_map.get(com, None)
            if com is None:
                raise ValueError("Invalid Command String")
        if com not in _command_valids:
            raise ValueError("Invalid Command Number")

        if com in _command_has_data and inp is None:
            raise ValueError(f"Input needed for 0x{com:02x}")

        packed_command = com | 1 << 8 | (inp << 16 if inp is not None else 0xffff0000)
        self.write(0x0, packed_command)

        while timeout > 0:
            rd = self.read(0x4)
            if rd & 0x1:
                break
            timeout -= 1
        if timeout == 0:
            return -1

        if com in _command_has_reply:
            while timeout > 0:
                rd = self.read(0x4)
                if rd & 0x100:
                    return (rd >> 16) & 0xffff
                timeout -= 1
            return -1
        return None

    def read_serial(self):
        msb = self.command("READ_SERIAL_MSB")
        lsb = self.command("READ_SERIAL_LSB")
        return (msb << 16 | lsb) if msb != -1 and lsb != -1 else -1

    def read_channel_status(self, channel):
        if channel > 255:
            raise ValueError("Channel should be between 0 and 255")
        if self.command("SET_CHANNEL", channel) == -1:
            return -1
        return self.command("CHANNEL_STATUS")

    def disable_channel(self, channel):
        if channel > 255:
            raise ValueError("Channel should be between 0 and 255")
        if self.command("SET_CHANNEL", channel) == -1:
            return -1
        return self.command("CHANNEL_CONTROL", 1)

    def enable_channel(self, channel):
        if channel > 255:
            raise ValueError("Channel should be between 0 and 255")
        if self.command("SET_CHANNEL", channel) == -1:
            return -1
        return self.command("CHANNEL_CONTROL", 0)

    def scan_all_channels(self):
        self.disabled_channel_list = [i for i in range(256) if self.read_channel_status(i) == 1]
        self.num_disabled = len(self.disabled_channel_list)

    def set_clock(self, clk):
        return self.command("SET_CLOCK", int(clk / 5))

    def get_clock(self):
        clk = self.command("GET_CLOCK")
        return clk * 5 if clk != -1 else -1

    def set_threshold(self, threshold):
        return self.command("SET_THRESHOLD", int(threshold * 1023.0 / 200.0))

    def get_threshold(self):
        threshold = self.command("GET_THRESHOLD")
        return threshold * 200.0 / 1023.0 if threshold != -1 else -1

    def get_temperature(self):
        return self.command("GET_TEMPERATURE")

def parse_event_data(event_data):
    return [((event & 0xffffffff00000000) >> 32, (event & 0x00000000ff000000) >> 24,
             (event & 0x0000000000ff0000) >> 16, (event & 0x000000000000ffff) >> 0) for event in event_data]

# Main execution
if __name__ == "__main__":
    ov = Overlay("./overlays/test_2det_commanding.bit")
    czt1 = ov.AXI_CZT_AXIS_0.AXI_CZT_Controller
    czt0 = ov.AXI_CZT_AXIS_1.AXI_CZT_Controller
    reset_pl = ov.Reset_system.reset_gpio.channel1[0].on
    dma_channel = ov.DMA.axi_dma_0.recvchannel
    wr_data_count = ov.AXIS_Combine.axi_gpio_0.channel1.read
    rd_data_count = ov.AXIS_Combine.axi_gpio_0.channel2.read

    print(czt0.read_serial(), czt1.read_serial())
    czt1.command("BREAK")
    czt0.command("BREAK")
    reset_pl()

    test_reply_commands = ['READ_SERIAL_LSB', 'READ_SERIAL_MSB', 'READ_FIRMWARE_VERSION',
                           'READ_MODULE_VERSION', 'EEPROM_CHECKSUM', 'GET_CLOCK', 'GET_EMULATOR',
                           'GET_GPIO', 'GET_PEAKING_TIME', 'GET_THRESHOLD', 'READ_STATUS', 'READ_TEMPERATURE']
    print("Command                    CZT 0  CZT 1")
    for comm in test_reply_commands:
        print(f"{comm:.<25}{czt0.command(comm):>7}{czt1.command(comm):>7}")

    czt1.command("EVENT_ON")
    czt0.command("EVENT_ON")
    print(dma_channel.idle)
    print(wr_data_count(), rd_data_count())

    czt0.command("EVENT_OFF")
    czt1.command("EVENT_OFF")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pynq import allocate

# Initialize buffers
event_data_buffer = allocate(shape=(128,), dtype=np.uint64)
main_buffer = []

# Constants
N_PACKETS = 2  # Number of packets to collect (0.6s data)

# Reset PL (commented out for now)
# reset_pl()

# Start DMA channel and enable events
dma_channel.start()
czt0.command("EVENT_ON")
czt1.command("EVENT_ON")

# Collect data packets
for _ in range(N_PACKETS):
    dma_channel.wait()
    main_buffer.extend(event_data_buffer.tolist())
    dma_channel.transfer(event_data_buffer)

# Stop events and issue break commands
czt1.command("EVENT_OFF")
czt0.command("EVENT_OFF")
czt1.command("BREAK")
czt0.command("BREAK")

# Parse event data
parse_main_buffer = parse_event_data(main_buffer)

# Separate data for detector 0 and detector 1
det0 = [x for x in parse_main_buffer if x[1] == 0]
det1 = [x for x in parse_main_buffer if x[1] == 1]

# Extract timestamps, pixels, and energy for each detector
times_det0 = [x[0] for x in det0]
pixels_det0 = [x[2] for x in det0]
energy_det0 = [x[3] for x in det0]

times_det1 = [x[0] for x in det1]
pixels_det1 = [x[2] for x in det1]
energy_det1 = [x[3] for x in det1]

# Generate pixel histograms
pixhist_det0 = np.bincount(pixels_det0, minlength=256)
pixhist_det1 = np.bincount(pixels_det1, minlength=256)

# Plot heatmaps for detector 0
plt.figure(figsize=(12, 12))
plt.title('Detector 0')
sns.heatmap(pixhist_det0.reshape((16, 16)), cmap="icefire", linewidths=1, annot=True, fmt=".0f")
print(f"Number of pixels that gave data (Detector 0): {len(np.unique(pixels_det0))}")
print(f"Number of disabled pixels (Detector 0): {czt0.num_disabled}")

# Plot heatmaps for detector 1
plt.figure(figsize=(12, 12))
plt.title('Detector 1')
sns.heatmap(pixhist_det1.reshape((16, 16)), cmap="icefire", linewidths=1, annot=True, fmt=".0f")
print(f"Number of pixels that gave data (Detector 1): {len(np.unique(pixels_det1))}")
print(f"Number of disabled pixels (Detector 1): {czt1.num_disabled}")

# Plot energy histograms
plt.figure(figsize=(14, 7))
plt.hist(energy_det0, bins=range(0, 4096, 10), alpha=0.5, color='b', label='Detector 0', density=True)
plt.hist(energy_det1, bins=range(0, 4096, 10), alpha=0.5, color='r', label='Detector 1', density=True)
plt.legend()
plt.yticks([])
plt.xlabel("PHA")
plt.ylabel("Normalized Counts")
plt.show()

# Save main buffer to a file
with open("2dets_bkg_iitb_20230609_stcreport.txt", "w") as file:
    file.write(str(main_buffer))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pynq import allocate
from time import time  # For performance measurement

# Initialize buffers
event_data_buffer = allocate(shape=(128,), dtype=np.uint64)
main_buffer = []

# Constants
N_PACKETS = 2  # Number of packets to collect (0.6s data)

# Start DMA channel and enable events
dma_channel.start()
czt0.command("EVENT_ON")
czt1.command("EVENT_ON")

# Collect data packets
start_time = time()
for _ in range(N_PACKETS):
    dma_channel.wait()
    main_buffer.extend(event_data_buffer.tolist())  # Avoid copying if possible
    dma_channel.transfer(event_data_buffer)
print(f"Data collection time: {time() - start_time:.2f} seconds")

# Stop events and issue break commands
czt1.command("EVENT_OFF")
czt0.command("EVENT_OFF")
czt1.command("BREAK")
czt0.command("BREAK")

# Parse event data
start_time = time()
parse_main_buffer = parse_event_data(main_buffer)
print(f"Event parsing time: {time() - start_time:.2f} seconds")

# Separate data for detector 0 and detector 1
start_time = time()
parse_main_buffer = np.array(parse_main_buffer)  # Convert to NumPy array for vectorized operations
det0 = parse_main_buffer[parse_main_buffer[:, 1] == 0]
det1 = parse_main_buffer[parse_main_buffer[:, 1] == 1]
print(f"Data separation time: {time() - start_time:.2f} seconds")

# Extract timestamps, pixels, and energy for each detector
start_time = time()
times_det0 = det0[:, 0]
pixels_det0 = det0[:, 2]
energy_det0 = det0[:, 3]

times_det1 = det1[:, 0]
pixels_det1 = det1[:, 2]
energy_det1 = det1[:, 3]
print(f"Data extraction time: {time() - start_time:.2f} seconds")

# Generate pixel histograms
start_time = time()
pixhist_det0 = np.bincount(pixels_det0, minlength=256)
pixhist_det1 = np.bincount(pixels_det1, minlength=256)
print(f"Histogram generation time: {time() - start_time:.2f} seconds")

# Plot heatmaps for detector 0
plt.figure(figsize=(12, 12))
plt.title('Detector 0')
sns.heatmap(pixhist_det0.reshape((16, 16)), cmap="icefire", linewidths=1, annot=True, fmt=".0f")
print(f"Number of pixels that gave data (Detector 0): {len(np.unique(pixels_det0))}")
print(f"Number of disabled pixels (Detector 0): {czt0.num_disabled}")

# Plot heatmaps for detector 1
plt.figure(figsize=(12, 12))
plt.title('Detector 1')
sns.heatmap(pixhist_det1.reshape((16, 16)), cmap="icefire", linewidths=1, annot=True, fmt=".0f")
print(f"Number of pixels that gave data (Detector 1): {len(np.unique(pixels_det1))}")
print(f"Number of disabled pixels (Detector 1): {czt1.num_disabled}")

# Plot energy histograms
plt.figure(figsize=(14, 7))
plt.hist(energy_det0, bins=range(0, 4096, 10), alpha=0.5, color='b', label='Detector 0', density=True)
plt.hist(energy_det1, bins=range(0, 4096, 10), alpha=0.5, color='r', label='Detector 1', density=True)
plt.legend()
plt.yticks([])
plt.xlabel("PHA")
plt.ylabel("Normalized Counts")
plt.show()

# Save main buffer to a file
start_time = time()
with open("2dets_bkg_iitb_20230609_stcreport.txt", "w") as file:
    file.write(str(main_buffer))
print(f"File saving time: {time() - start_time:.2f} seconds")